In [1]:
from ssd.mobilenetv1_ssd import create_mobilenetv1_ssd
from ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite
from ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite

import sys
import torch.onnx
# from caffe2.python.onnx.backend import Caffe2Backend as c2
import onnx


In [10]:
net_type = 'mb2-ssd-lite'
model_path = './weights/mb2-ssd-lite-Epoch-0-Loss-6.8361222943952.pth'

label_path = './weights/voc-model-labels.txt'

class_names = [name.strip() for name in open(label_path).readlines()]
num_classes = len(class_names)
device_id=0 
if net_type == 'vgg16-ssd':
    net = create_vgg_ssd(len(class_names), is_test=True)
elif net_type == 'mb1-ssd':
    net = create_mobilenetv1_ssd(len(class_names), is_test=True,device_id=0 )
elif net_type == 'mb1-ssd-lite':
    net = create_mobilenetv1_ssd_lite(len(class_names), is_test=True)
elif net_type == 'mb2-ssd-lite':
    net = create_mobilenetv2_ssd_lite(len(class_names), is_test=True,device_id=0)
elif net_type == 'sq-ssd-lite':
    net = create_squeezenet_ssd_lite(len(class_names), is_test=True)
else:
    print("The net type is wrong. It should be one of vgg16-ssd, mb1-ssd and mb1-ssd-lite.")
    sys.exit(1)
DEVICE = torch.device("cuda:%d"%device_id if torch.cuda.is_available()  else "cpu")
net = net.to(DEVICE)
net.load(model_path)
net.eval()

model_path = f"weights/{net_type}.onnx"
init_net_path = f"models/{net_type}_init_net.pb"
init_net_txt_path = f"models/{net_type}_init_net.pbtxt"
predict_net_path = f"models/{net_type}_predict_net.pb"
predict_net_txt_path = f"models/{net_type}_predict_net.pbtxt"

dummy_input = torch.randn(1, 3, 300, 300)

dummy_input = dummy_input.to(DEVICE)

In [11]:
dummy_input.dtype

torch.float32

In [12]:
net

SSD(
  (base_net): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Co

In [13]:
net.save('f_mb1_ssd.pth')
torch.onnx.export(net, dummy_input, 'f_mb1_ssd.onnx', verbose=True, output_names=['scores', 'boxes'])

graph(%input.1 : Float(1, 3, 300, 300),
      %base_net.0.0.weight : Float(32, 3, 3, 3),
      %base_net.0.1.weight : Float(32),
      %base_net.0.1.bias : Float(32),
      %base_net.0.1.running_mean : Float(32),
      %base_net.0.1.running_var : Float(32),
      %base_net.0.1.num_batches_tracked : Long(),
      %base_net.1.conv.0.weight : Float(32, 1, 3, 3),
      %base_net.1.conv.1.weight : Float(32),
      %base_net.1.conv.1.bias : Float(32),
      %base_net.1.conv.1.running_mean : Float(32),
      %base_net.1.conv.1.running_var : Float(32),
      %base_net.1.conv.1.num_batches_tracked : Long(),
      %base_net.1.conv.3.weight : Float(16, 32, 1, 1),
      %base_net.1.conv.4.weight : Float(16),
      %base_net.1.conv.4.bias : Float(16),
      %base_net.1.conv.4.running_mean : Float(16),
      %base_net.1.conv.4.running_var : Float(16),
      %base_net.1.conv.4.num_batches_tracked : Long(),
      %base_net.2.conv.0.weight : Float(96, 16, 1, 1),
      %base_net.2.conv.1.weight : Float(

In [6]:
torch.onnx.export(net, dummy_input, model_path, verbose=True, output_names=['scores', 'boxes'])

graph(%input.1 : Float(1, 3, 300, 300),
      %base_net.0.0.weight : Float(32, 3, 3, 3),
      %base_net.0.1.weight : Float(32),
      %base_net.0.1.bias : Float(32),
      %base_net.0.1.running_mean : Float(32),
      %base_net.0.1.running_var : Float(32),
      %base_net.0.1.num_batches_tracked : Long(),
      %base_net.1.0.weight : Float(32, 1, 3, 3),
      %base_net.1.1.weight : Float(32),
      %base_net.1.1.bias : Float(32),
      %base_net.1.1.running_mean : Float(32),
      %base_net.1.1.running_var : Float(32),
      %base_net.1.1.num_batches_tracked : Long(),
      %base_net.1.3.weight : Float(64, 32, 1, 1),
      %base_net.1.4.weight : Float(64),
      %base_net.1.4.bias : Float(64),
      %base_net.1.4.running_mean : Float(64),
      %base_net.1.4.running_var : Float(64),
      %base_net.1.4.num_batches_tracked : Long(),
      %base_net.2.0.weight : Float(64, 1, 3, 3),
      %base_net.2.1.weight : Float(64),
      %base_net.2.1.bias : Float(64),
      %base_net.2.1.running_

In [7]:
import cv2
import onnxruntime
print(cv2.__version__)

4.1.2


In [14]:
# cv2读取失败 暂时还不知道怎么解决  使用is_test=True 到处的会成功读取
onnx_net = cv2.dnn.readNetFromONNX('f_mb1_ssd.onnx')

error: OpenCV(4.1.2) /io/opencv/modules/dnn/src/onnx/onnx_importer.cpp:101: error: (-211:One of the arguments' values is out of range) Input is out of OpenCV 32S range in function 'convertInt64ToInt32'


In [8]:
cv2.dnn.readNetFromTorch('f_mb1_ssd.pth')

error: OpenCV(4.1.2) /io/opencv/modules/dnn/src/torch/torch_importer.cpp:1022: error: (-213:The function/feature is not implemented) Unsupported Lua type in function 'readObject'


In [17]:
onnx_net

<dnn_Net 0x7f01850063d0>

In [26]:
image = cv2.imread('p1.jpg')
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
onnx_net.setInput(blob)
detections= onnx_net.forward()

In [19]:
detections

array([[[ 0.37891603,  1.2460072 , -2.5606096 , -2.493836  ],
        [-0.1395236 ,  0.87926835, -1.2438047 , -0.67212695],
        [ 0.91649723,  0.33521783, -3.6832418 , -2.5664163 ],
        ...,
        [-1.5009778 ,  0.5787486 ,  0.55361694, -1.0585183 ],
        [-0.9001981 ,  1.0768944 , -1.2175984 ,  1.6722215 ],
        [-1.8517532 ,  0.5306453 ,  1.3910294 , -1.0777861 ]]],
      dtype=float32)

In [28]:
detections[0][0].shape

(4,)

In [21]:
import numpy as np
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
for i in np.arange(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.2:
        idx = int(detections[0, 0, i, 1])
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        if CLASSES[idx] == 'person':   #如果检测到人，根据输入图片缩放x，y然后返回人矩形框坐标
            width_scale = w/300
            hight_scale = h/300
            center_x =  width_scale*(startX+endX)/2
            center_y = hight_scale*(startY+endY)/2
            w = width_scale*(endX-startX)
            h =hight_scale*(endY-startY)
            return_list.append([center_x,center_y,w,h])
            print('person')


IndexError: too many indices for array

In [8]:
import onnxruntime
import numpy as np
from onnxruntime.datasets import get_example

In [14]:
sess = onnxruntime.InferenceSession("mb1_ssd.onnx")

In [15]:
input_name = sess.get_inputs()[0].name
print("Input name  :", input_name)
input_shape = sess.get_inputs()[0].shape
print("Input shape :", input_shape)
input_type = sess.get_inputs()[0].type
print("Input type  :", input_type)

Input name  : input.1
Input shape : [1, 3, 300, 300]
Input type  : tensor(float)


In [25]:

output_name = sess.get_outputs()[1].name
print("Output name  :", output_name)  
output_shape = sess.get_outputs()[1].shape
print("Output shape :", output_shape)
output_type = sess.get_outputs()[0].type
print("Output type  :", output_type)

Output name  : boxes
Output shape : [1, 3000, 4]
Output type  : tensor(float)


In [28]:
outname = [output.name for output in sess.get_outputs()]
print("outputs name:",outname)


outputs name: ['scores', 'boxes']


In [32]:
image = cv2.imread('p1.jpg')

In [36]:
i = cv2.resize(image,(300,300))
print(i.shape)

(300, 300, 3)


In [37]:
i = np.expand_dims(i,axis=0)
print(i.shape)

(1, 300, 300, 3)


In [38]:
i = np.transpose(i, [0,3,1,2]) 
print(i.shape)

(1, 3, 300, 300)


In [41]:
i.dtype

dtype('uint8')

In [43]:
result[0][0][0]

In [46]:
result[0][0][0]

array([9.7291046e-01, 1.4906304e-04, 1.8564858e-04, 4.3731893e-04,
       6.0622511e-04, 2.7387606e-03, 9.9050914e-05, 3.7520396e-04,
       4.0574075e-04, 1.1100336e-03, 1.1617724e-04, 6.8446656e-04,
       4.5141290e-04, 1.8528386e-04, 1.0642443e-04, 1.3091291e-02,
       2.1112843e-03, 1.2555563e-04, 3.6207089e-04, 1.2455645e-04,
       3.6240600e-03], dtype=float32)

In [45]:
result[1].shape

(1, 3000, 4)

In [ ]:
res